In [ ]:
## Use shapefiles to extract TWS within the basin

In [ ]:
## Applications of satellite gravimetry 

In [ ]:
Try to create 3 time series for Amazon, Nile and Ganga River Basins

Explain what you see in the time series. Do you see specific extreme events happening?

Can you see a difference between the different locations? Try to find an explanation of what you're seeing.

Can you think of another application of the TWS time series? 

What are the impacts of climate change on water storage?


How to monitor groundwater from GRCAE